Run the cell below if you are using Google Colab to mount your Google Drive in your Colab instance. Adjust the path to the files in your Google Drive as needed if it differs.

If you do not use Google Colab, running the cell will simply do nothing, so do not worry about it.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd 'drive/My Drive/Colab Notebooks/05_Classification'
except ImportError as e:
    pass

## Exercise 5: Classification

### 5.1. Parameter optimization
In Exercise 4.2 we have used the German credit data set from the UCI data set library (http://archive.ics.uci.edu/ml/index.html), which describes the customers of a bank with respect to whether they should get a bank credit or not. The data set is provided as credit-g.arff file in ILIAS.

#### 5.1.1.	(recap) Go back to the results of exercise 4.2.4. Re-run the classifiers with their default parameter settings.
- Used the 10-fold validation approach.
- Balanced the training set multiplying the “bad customer” examples. 
- Evaluated the results, setting up your cost matrix to ((0,100)(1,0)) – that is, you assumed you will lose 1 Unit if you refuse a credit to a good customer, but that you lose 100 Units if you give a bad customer a credit.

Rerun your process to get the performance results. What were the default parameters of the Decision Tree?

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
import pandas as pd
from scipy.io import arff
from sklearn.preprocessing import LabelEncoder

# load the dataset
credit_arff_data, credit_arff_meta = arff.loadarff(open('credit-g.arff', 'r'))
credit_data = pd.DataFrame(credit_arff_data)

# select all columns of type object
columns_with_binary_strings = credit_data.select_dtypes('object').columns.values

# decode the values of these columns using utf-8
credit_data[columns_with_binary_strings] = credit_data[columns_with_binary_strings].apply(lambda x: x.str.decode("utf-8"))
credit_target = credit_data['class']
credit_data = credit_data.drop(columns='class')

label = LabelEncoder()
credit_target = label.fit_transform(credit_target)
label_names=['bad','good']
label_order=label.transform(label_names)

credit_data.head()

Then, we set up a pipeline and evaluate it using cross validation:

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer

numeric_features = ['duration', 'credit_amount', 'installment_commitment', 'residence_since', 'age', 'existing_credits', 'num_dependents']
categorical_features = ['credit_history', 'purpose', 'personal_status', 'other_parties', 'property_magnitude', 'other_payment_plans', 'housing', 'job', 'own_telephone', 'foreign_worker']
ordinal_features = [ 'checking_status', 'savings_status', 'employment']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features),
        ('ord', OrdinalEncoder(categories=[
            [ 'no checking', '<0', '0<=X<200', '>=200' ],
            [ 'no known savings', '<100', '100<=X<500', '500<=X<1000', '>=1000' ],
            [ 'unemployed', '<1', '1<=X<4', '4<=X<7', '>=7' ]
        ]), ordinal_features)])

# define a pipeline that includes a preprocessing step, an oversampling balancing step and a decision tree classifier
#TODO: INSERT YOUR CODE HERE!

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# get the predictions from the pipeline using the created StratifiedKFold instance
#TODO: INSERT YOUR CODE HERE!

def cost_function(y_true, y_pred): 
    cm = confusion_matrix(y_true, y_pred, labels=label_order)
    return cm[0][1] * 100 + cm[1][0] * 1

# create the confusion matrix, calculate the cost with the cost_function and calculate the accuracy
#TODO: INSERT YOUR CODE HERE!

print("Decision Tree with accuracy of {} and cost {}".format(acc, cost))
plot_confusion_matrix(cm, classes=label_names, title='Decision Tree Classifier')
plt.show()
print(classification_report(credit_target, prediction, target_names=label_names))

In [ ]:
from sklearn import tree
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt

# fit the pipeline to the dataset
pipeline.fit(credit_data, credit_target)

estimator = pipeline.named_steps['estimator']
pre = pipeline.named_steps['preprocessing']
feature_names = numeric_features + list(pre.named_transformers_['cat'].get_feature_names(categorical_features)) + ordinal_features

# plot the decision tree
plt.figure(figsize=(20,10))
tree.plot_tree(estimator,
               feature_names=feature_names,
                class_names=label_names)
plt.show()

In [ ]:
# check the default parameters of the decision tree
tree.DecisionTreeClassifier()

#### 5.1.2.	Now try to find a more optimal configuration for the Decision Tree. What is the best parameter setting and what is the cross-validated performance of this appraoch?

Use the GridSearchCV from scikit learn. Try the following parameters of the Decision Tree:
- criterion: ['gini', 'entropy']
- 'max_depth': [2, 3, 4, 5, None]
- 'min_samples_split' :[2,3,4,5]

What is the best configuration for the data set and the classification approach? 

Note: The grid search can take some time. You can use the ```n_jobs=-1``` parameter setting for the ```cross_val_predict()``` function to enable parallel processing (all CPU cores will be used).

In [ ]:
from sklearn.model_selection import GridSearchCV

# define the parameter grid
#TODO: INSERT YOUR CODE HERE

# define the folds for the cross validation
stratified_10_fold_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# create a scorer for the grid search
cost_score = make_scorer(cost_function, greater_is_better=False)

# create the grid search estimator
#TODO: INSERT YOUR CODE HERE
#grid_search_estimator = ...

# cross-validate
#TODO: INSERT YOUR CODE HERE
#prediction = ...

# calculate costs
cm = confusion_matrix(credit_target, prediction, labels=label_order)
cost = cost_function(credit_target, prediction)
acc = accuracy_score(credit_target, prediction)

print("Optimised Decision Tree with accuracy of {} and cost {}".format(acc, cost))
plot_confusion_matrix(cm, classes=label_names, title='Decision Tree Classifier')
plt.show()
print(classification_report(credit_target, prediction, target_names=label_names))

# fit the grid search (= determine the optimal parameters)
#TODO: INSERT YOUR CODE HERE
print("Optimised Parameters: {}".format(grid_search_estimator.best_params_))

#### 5.1.4.	How does the optimal decision tree differ from the one you have learned in 4.2.4?
Plot the optimised tree!

In [ ]:
estimator = grid_search_estimator.best_estimator_.named_steps['estimator']
pre = grid_search_estimator.best_estimator_.named_steps['preprocessing']
feature_names = numeric_features + list(pre.named_transformers_['cat'].get_feature_names(categorical_features)) + ordinal_features

plt.figure(figsize=(20,10))
tree.plot_tree(estimator,
               feature_names=feature_names,
                class_names=label_names)
plt.show()

### 5.2. Open Competition: Finding rich Americans
The Adult data set from the UCI data set library (http://archive.ics.uci.edu/ml/datasets/Adult) describes 48842 persons from the 1994 US Census. The data set is provided as adult.arff file on the website of this course. 

Your task is to find a good classifier for determining whether a person earns over 50.000 $ 
a year. Beside of being accurate, your classifier should also have balanced precision and recall.

To evaluate your classifiers use train_test_split validation (test_size=0.2, random_state=42).
In order to find the best classifier, you may experiment with:
1.	different algorithms
2.	different parameter settings
3.	the balance of the two classes in the data set
4.	the set of attributes that are used or not used
5.	other preprocessing techniques

In order to increase your understanding of the data set, you might want to visualize different attributes or attribute combinations.

In [ ]:
# load the dataset
adult_arff_data, adult_arff_meta = arff.loadarff(open('adult.arff', 'r'))
adult = pd.DataFrame(adult_arff_data)
adult = adult.applymap(lambda x: x.decode('utf8').replace("'", "") if hasattr(x, 'decode') else x)

adult_target = np.array(adult['class'])
adult_data = adult.drop('class', axis=1)
adult_data.head()

In [ ]:
# Create train/test split
from sklearn.model_selection import train_test_split
adult_data_train, adult_data_test, adult_target_train, adult_target_test = train_test_split(
    adult_data, adult_target, test_size=0.2, random_state=42)

In [ ]:
# define a Column Transformer to handle preprocessing
# TODO: INSERT YOUR CODE HERE!

In [ ]:
# test some classification methods as baseline
# TODO: INSERT YOUR CODE HERE!

In [ ]:
# plot the class distribution
class_dist = pd.Series(adult_target_train).value_counts()
plt.bar(class_dist.index, class_dist)
plt.show()

In [ ]:
# try balancing the dataset: add an additional step to the pipeline
# TODO: INSERT YOUR CODE HERE!

# create a grid search instance and find the optimal parameters for a decision tree classifier
# TODO: INSERT YOUR CODE HERE!

In [ ]:
# find the optimal parameters for a KNN classifier
# TODO: INSERT YOUR CODE HERE!

In [ ]:
# select the best estimator and the optimal hyperparameters from the above results and evaluate on the test(unseen) data
# TODO: INSERT YOUR CODE HERE!

# plot the confusion matrix and print the classification report
cm = confusion_matrix(adult_target_test, prediction)
plot_confusion_matrix(cm, classes=unique_labels(adult_target_test), title="Results on Test Set")
print(classification_report(adult_target_test, prediction))

# ChatGPT Bonus Exercises
Reminder: Do not take the answers of ChatGPT at face value! Always cross-check with lecture slides, literature and/or the teaching staff!

### C.1. Discuss about Hyperparameter Optimization and (Nested) Cross Validation
* Use ChatGPT to discuss what hyperparameter optimization is and why it is important for achieving good results. Can hyperparameter optimization help with the problem of overfitting? Ask ChatGPT to give you an example of overcoming overfitting in Decision Trees by performing hyperparameter optimization.
* Discuss with ChatGPT what the difference is between nested cross validation and cross validation. What are the factors that determine the number of folds to use for nested cross validation? Is it better to use a higher number of folds?

### C.2. Learn about Naïve Bayes and Feature Selection
* Ask ChatGPT how Naïve Bayes is used for classification, its advantages and disadvantages and how you can overcome these disadvantages. Ask ChatGPT to give you a small dataset and an unseen example for a pen and paper exercise on how Naïve Bayes is used to classify the unseen example. Afterwards, ask ChatGPT for Python code to load the proposed dataset and classify the unseen example using Naïve Bayes.

In [ ]:
# put your code here

* Ask ChatGPT to list some feature selection methods and to give you an explanation for each of the methods. Afterwards, ask ChatGPT to give you Python code for performing feature selection on the “Iris” dataset. Afterwards, ask ChatGPT to give you Python code for testing the performance of Naïve Bayes on the original as well as on the reduced dataset.

In [ ]:
# put your code here

### C.3. Self-Assessment
* Ask ChatGPT to give you some multiple-choice questions for graduate students about Support Vector Machines and Naïve Bayes and solve the questions. Request the correct answers and compare with your own answers.